In [130]:
from pydantic import BaseModel
from enum import Enum
import json
import random
from git import Repo, Commit


In [7]:
for i in range(256):
    print(chr(i).encode("utf-8"))

b'\x00'
b'\x01'
b'\x02'
b'\x03'
b'\x04'
b'\x05'
b'\x06'
b'\x07'
b'\x08'
b'\t'
b'\n'
b'\x0b'
b'\x0c'
b'\r'
b'\x0e'
b'\x0f'
b'\x10'
b'\x11'
b'\x12'
b'\x13'
b'\x14'
b'\x15'
b'\x16'
b'\x17'
b'\x18'
b'\x19'
b'\x1a'
b'\x1b'
b'\x1c'
b'\x1d'
b'\x1e'
b'\x1f'
b' '
b'!'
b'"'
b'#'
b'$'
b'%'
b'&'
b"'"
b'('
b')'
b'*'
b'+'
b','
b'-'
b'.'
b'/'
b'0'
b'1'
b'2'
b'3'
b'4'
b'5'
b'6'
b'7'
b'8'
b'9'
b':'
b';'
b'<'
b'='
b'>'
b'?'
b'@'
b'A'
b'B'
b'C'
b'D'
b'E'
b'F'
b'G'
b'H'
b'I'
b'J'
b'K'
b'L'
b'M'
b'N'
b'O'
b'P'
b'Q'
b'R'
b'S'
b'T'
b'U'
b'V'
b'W'
b'X'
b'Y'
b'Z'
b'['
b'\\'
b']'
b'^'
b'_'
b'`'
b'a'
b'b'
b'c'
b'd'
b'e'
b'f'
b'g'
b'h'
b'i'
b'j'
b'k'
b'l'
b'm'
b'n'
b'o'
b'p'
b'q'
b'r'
b's'
b't'
b'u'
b'v'
b'w'
b'x'
b'y'
b'z'
b'{'
b'|'
b'}'
b'~'
b'\x7f'
b'\xc2\x80'
b'\xc2\x81'
b'\xc2\x82'
b'\xc2\x83'
b'\xc2\x84'
b'\xc2\x85'
b'\xc2\x86'
b'\xc2\x87'
b'\xc2\x88'
b'\xc2\x89'
b'\xc2\x8a'
b'\xc2\x8b'
b'\xc2\x8c'
b'\xc2\x8d'
b'\xc2\x8e'
b'\xc2\x8f'
b'\xc2\x90'
b'\xc2\x91'
b'\xc2\x92'
b'\xc2\x93'
b'\xc2\x94'
b'\xc2\x95'
b'

In [65]:
RESERVED_STRING = "#" * 10

def make_paragraph(header: str, content: str) -> str:
    return  RESERVED_STRING + header + RESERVED_STRING + "\n" + content + "\n"

class RecipeKind(Enum):
    CREME_DOLCI = "creme_dolci"
    SUGHI = "sughi"
    ANTIPASTI = "antipasti"
    TORTE_SALATE = "torte_salate"
    SALSE = "salse"
    PASTE_BASE = "paste_base"
    SECONDI_DI_CARNE = "secondi_di_carne"
    DOLCI = "dolci"
    PANE_CO = "pane_&_co"
    SECONDI_DI_PESCE = "secondi_di_pesce"
    VERDURE = "verdure"

class GitFriendlyPersistable(BaseModel):
    def serialize(self) -> str:
        s_raw = self._serialize_raw()
        return s_raw + make_paragraph("hash", str(hash(s_raw)))

    def _serialize_raw(self) -> str:
        s = ""
        def make_paragraph(header: str, content: str) -> str:
            return  RESERVED_STRING + header + RESERVED_STRING + "\n" + content + "\n"

        for field_name, field in self:
            if isinstance(field, Enum):
                s += make_paragraph(field_name, field.value)
            else:
                s += make_paragraph(field_name, field)
        return s
    

    @classmethod
    def unserialize(cls, data: str) -> 'Recipe':
        content = data.split(RESERVED_STRING)
        assert content.pop(0) == ""
        output_dict = {}
        key = None
        for row in content:
            if key is None:
                key = row
            else:
                output_dict[key] = row[1:-1]
                key = None
        h = output_dict.pop("hash")
        output = cls(**output_dict)
        assert hash(output._serialize_raw()) == int(h)
        return cls(**output_dict)


class Recipe(GitFriendlyPersistable):
    title: str
    category: RecipeKind
    ingredients: str
    steps: str
    notes: str


In [79]:
rcps = json.load(open("reciepes.json"))
recipes = []
for rec in rcps:
    for k in rec:
        if isinstance(rec[k], list):
            rec[k] = "* " + "\n* ".join(rec[k])
    if "notes" not in rec:
        rec["notes"] = ""
    recipes.append(Recipe(**rec))

In [98]:
idxs = []
for idx, rec in enumerate(recipes):
    idx_key = "".join([random.sample("abcdefghijklmnopqrstuvwxyz0123456789", k = 1)[0] for _ in range(10)])
    assert idx_key not in idxs
    idxs.append(idx_key)
    with open(f"data/{idx_key}.recipe", "w") as f:
        f.write(rec.serialize())

In [135]:
import os


class GitDB:
    def __init__(self, folder_path: str) -> None:
        self.folder_path: str = folder_path
        self.cache: dict[str, Recipe] = self.init_cache()
        self.repo: Repo = Repo(folder_path)

    def init_cache(self) -> dict[str, Recipe]:
        out = {}
        for filename in os.listdir(self.folder_path):
            if len(filename) > 7 and filename[-7:] == ".recipe":
                idx = filename[:-7]
                with open(os.path.join(self.folder_path, filename)) as f:
                    out[idx] = Recipe.unserialize(f.read())
        return out

    def get_recipe(self, idx: str) -> Recipe:
        return self.cache[idx]

    def set_recipe(self, idx: str, recipe: Recipe, user: str) -> None:
        is_new = idx in self.cache
        self.cache[idx] = recipe
        path = os.path.join(self.folder_path, idx + ".recipe")
        with open(path, "w") as f:
            f.write(self.cache[idx].serialize())
        self.repo.index.add([path])
        self.repo.index.commit(f"user {user} {'created' if is_new else 'edited'} {idx}")

    def check_status(self):
        return self.cache == self.init_cache()



In [140]:
db = GitDB("./data")

In [143]:
db.repo.index.add(["./../data/wq1i9pvm8k.recipe"])

[(100644, c1d7e2e062f4a943b1a0a02856079b516115b222, 0, ./../data/wq1i9pvm8k.recipe)]

In [144]:
is_new = False
db.repo.index.commit(f"user io {'created' if is_new else 'edited'} {idx}")

<git.Commit "3fc120ed047c2415b8a54b5a7513de3bd6703196">

In [145]:
repo = Repo("./data")

In [146]:
repo.untracked_files

[]

In [141]:
r = db.get_recipe("wq1i9pvm8k")
r2 = Recipe(title=r.title, category=r.category, ingredients=r.ingredients, steps=r.steps, notes="molto buona")
db.set_recipe("wq1i9pvm8k", r2, user="io")

FileNotFoundError: [Errno 2] No such file or directory: './data/wq1i9pvm8k.recipe'

In [115]:
r.notes = "molto buona"

In [ ]:
db.set_recipe("wq1i9pvm8k", r, io)

In [108]:
db.cache

{'lgq19tt7uj': Recipe(title='torta_di_semolino', category=<RecipeKind.DOLCI: 'dolci'>, ingredients="* 200 gr semolino\n* 500 gr latte\n* 500 gr acqua\n* 350 gr ricotta\n* 250 gr zucchero\n* 4 uova\n* 40 gr burro\n* Scorza d'arancia\n* Vanillina\n* Sale\n* Zucchero a velo", steps="* In una pentola saldare il latte con l'acqua e la scorza d'arancia.\n* Prima che raggiunga il bollore rimuovere la scorza e aggiungere il burro. Farlo sciogliere quindi aggiungere a pioggia il semolino. Cuocere per 6-8 minuti fino a quando il composto si addensa.\n* In una ciotola montare le uova con lo zucchero e la vanillina fino ad ottenere una spuma. Incorporare poco per volta la ricotta e infine il semolino intiepidito.\n* Cuocere in uno stampo da 24 cm a 180 C° per circa 50'.\n* Lasciare raffreddare e spolverizzare con lo zucchero a velo.", notes=''),
 '6cjlbu8765': Recipe(title='pesce_finto', category=<RecipeKind.TORTE_SALATE: 'torte_salate'>, ingredients="* Gr 250 tonno sott'olio\n* 400 gr patate\n* 2

In [104]:
os.listdir("./data")[0][-7:]

'.recipe'